<a href="https://colab.research.google.com/github/agustin90-ush/langchain-ask-pdf/blob/main/Copy_of_LangChain_Working_with_LLM_Models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## HuggingFace

Hugging face provide two wrappers for LLM. 
1. Models hosted on HuggingFace Hub via API
2. Local pipelines (download models locally). 

Both works with: text2text-generation, text-generation



In [ ]:
!pip -q install langchain huggingface_hub transformers sentence_transformers accelerate bitsandbytes

In [ ]:
import os
os.environ['HUGGINGFACEHUB_API_TOKEN'] = 'HUGGINGFACEHUB_API_TOKEN'

## Use the HuggingFaceHub via API
- T5 Encoder-Decoder Model

In [ ]:
from langchain import PromptTemplate, HuggingFaceHub, LLMChain

template = """Question: {question}

Answer: Let's think step by step."""

prompt = PromptTemplate(template=template, input_variables=["question"])

In [ ]:
llm_chain = LLMChain(prompt=prompt, 
                     llm=HuggingFaceHub(repo_id="google/flan-t5-xl", 
                                        model_kwargs={"temperature":0, 
                                                      "max_length":64}))

In [ ]:
question = "What is the capital of England?"

print(llm_chain.run(question))

London is the capital of England. The final answer: London.


In [ ]:
question = "What is Area 51 famous for?"

print(llm_chain.run(question))

Area 51 is famous for being the location of the US government's secret space program. The US government has been constructing a space station in Area 51. The final answer: space station.


- Decoder Only Models
text-generation models

In [ ]:
# llm_chain = LLMChain(prompt=prompt, 
#                      llm=HuggingFaceHub(repo_id="EleutherAI/gpt-j-6b", 
#                                         model_kwargs={"temperature":0.1, 
#                                                       "max_length":100}))

In [ ]:
# llm_chain = LLMChain(prompt=prompt, 
#                      llm=HuggingFaceHub(repo_id="stabilityai/stablelm-tuned-alpha-7b", 
#                                         model_kwargs={"temperature":0, 
#                                                       "max_length":100}))

# question = "What is Area 51 famous for?"

# print(llm_chain.run(question))

## Local pipelines (download models locally)


- T5-Flan - Encoder-Decoder (Seq2seq model)

In [ ]:
from langchain.llms import HuggingFacePipeline
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline, AutoModelForSeq2SeqLM

model_id = 'google/flan-t5-small'
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForSeq2SeqLM.from_pretrained(model_id, load_in_8bit=True, device_map='auto')

pipeline = pipeline(
    "text2text-generation",
    model=model, 
    tokenizer=tokenizer, 
    max_length=128
)

local_llm = HuggingFacePipeline(pipeline=pipeline)


Overriding torch_dtype=None with `torch_dtype=torch.float16` due to requirements of `bitsandbytes` to enable model loading in mixed int8. Either pass torch_dtype=torch.float16 or don't pass this argument at all to remove this warning.



===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /usr/local/lib/python3.9/dist-packages/bitsandbytes/libbitsandbytes_cuda118.so
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching in backup paths...
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so.11.0
CUDA SETUP: Highest compute capability among GPUs detected: 7.5
CUDA SETUP: Detected CUDA version 118
CUDA SETUP: Loading binary /usr/local/lib/python3.9/dist-packages/bitsandbytes/libbitsandbytes_cuda118.so...


/usr/local/lib/python3.9/dist-packages/bitsandbytes/cuda_setup/main.py:145: UserWarning: /usr/lib64-nvidia did not contain ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] as expected! Searching further paths...
  warn(msg)
/usr/local/lib/python3.9/dist-packages/bitsandbytes/cuda_setup/main.py:145: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/sys/fs/cgroup/memory.events /var/colab/cgroup/jupyter-children/memory.events')}
  warn(msg)
/usr/local/lib/python3.9/dist-packages/bitsandbytes/cuda_setup/main.py:145: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('8013'), PosixPath('//172.28.0.1'), PosixPath('http')}
  warn(msg)
/usr/local/lib/python3.9/dist-packages/bitsandbytes/cuda_setup/main.py:145: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('--listen_host=172.28.0.12 --target_host=172.28

In [ ]:
# print(local_llm('What is the capital of England? '))

In [ ]:
llm_chain = LLMChain(prompt=prompt, 
                     llm=local_llm
                     )

question = "What is the capital of England?"

print(llm_chain.run(question))

England is the capital of England. The capital of England is London. So, the answer is London.


## Decoder Only Model - Usage
---



microsoft/DialoGPT-large

In [ ]:
model_id = "gpt2-medium"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id)

pipeline = pipeline(
    "text-generation", 
    model=model, 
    tokenizer=tokenizer, 
    max_length=100
)

local_llm = HuggingFacePipeline(pipeline=pipeline)